In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [6]:
DELETE_TABLES=True
if DELETE_TABLES:
    try:
        cmd="""DROP TABLE `%s`"""%osm_intersections_table_name
        cursor.execute(cmd)
    except Exception as e:
        print(e)
        pass
    cmd="""DROP TABLE `%s`"""%osm_nodes_table_name
    cursor.execute(cmd)
2

1051 (42S02): Unknown table 'dem_from_contours_lines.osm_intersections'


2

In [7]:
CREATE_TABLES=True
if CREATE_TABLES:
    cmd="""CREATE TABLE `%s`(
    `osm_id` BIGINT NOT NULL,
    `contour_line_id` INT,
    `geometry` POINT NOT NULL,
    `elevation` FLOAT,

    FOREIGN KEY (contour_line_id) REFERENCES %s(id),
    PRIMARY KEY (osm_id)
    );
        """%(osm_nodes_table_name,contours_lines_table_name)
    
    cursor.execute(cmd)



    
    cmd="""CREATE TABLE `%s`(
        `id` INT NOT NULL AUTO_INCREMENT,
        `osm_begin` BIGINT NOT NULL,
        `osm_end` BIGINT NOT NULL,
        `osm_key` INT NOT NULL,
        `contour_line_id` INT NOT NULL,
        `edge_coordinate` FLOAT NOT NULL,
        `geometry` POINT NOT NULL,
        `elevation` FLOAT NOT NULL,

        FOREIGN KEY (osm_begin) REFERENCES %s(osm_id),
        FOREIGN KEY (osm_end) REFERENCES %s(osm_id),
        FOREIGN KEY (contour_line_id) REFERENCES %s(id),
        UNIQUE (osm_begin,osm_end,osm_key,edge_coordinate),
        PRIMARY KEY (id)

          )
          ENGINE=InnoDB;
        """%(osm_intersections_table_name,osm_nodes_table_name,osm_nodes_table_name,contours_lines_table_name)
    cursor.execute(cmd)
    

2

2

In [8]:
t1=time.time()
G=build_graph(cursor)
max_depth=np.max([data['depth'] for _,data in G.nodes(data=True)])
t2=time.time()
print(t2-t1)

1.8465242385864258


In [9]:
t1=time.time()
place_name='Corse, France'
G_osm=ox.graph_from_place(place_name,network_type='drive')
G_osm=to_multi_graph(G_osm)
osm_crs=ox.graph_to_gdfs(G_osm,edges=False).estimate_utm_crs()
G_osm=ox.project_graph(G_osm,to_crs=osm_crs)
add_missing_geometries(G_osm)
t2=time.time()
print(t2-t1)
len(G_osm.nodes()),len(G_osm.edges())

67.59671115875244


(17435, 20792)

In [10]:
t1=time.time()
osm_edges_df=ox.graph_to_gdfs(G_osm,nodes=False,edges=True)
osm_edges_df['edge']=osm_edges_df.index
osm_edges_df=osm_edges_df.loc[:,['edge','geometry']]
osm_edges_df['number_of_points']=osm_edges_df['geometry'].apply(lambda ls:len(ls.coords))
#osm_edges_df=osm_edges_df.sort_values(by='length')
osm_edges_df=osm_edges_df.set_index(np.arange(len(osm_edges_df)))
osm_edges_df['id_edge']=osm_edges_df.index
t2=time.time()
print(t2-t1)
osm_edges_df

0.3589158058166504


,edge,geometry,number_of_points,id_edge
0,"(14468319, 15905397, 0)","LINESTRING (525532.710 4599879.701, 525543.715...",82,0
1,"(14468319, 1434053206, 0)","LINESTRING (525532.710 4599879.701, 525519.651...",4,1
2,"(14468319, 1434053161, 0)","LINESTRING (525532.710 4599879.701, 525532.752...",20,2
3,"(15905397, 1976904946, 0)","LINESTRING (526252.098 4600660.017, 526286.419...",12,3
4,"(15905397, 1976906939, 0)","LINESTRING (526252.098 4600660.017, 526253.327...",7,4
...,...,...,...,...
20787,"(9744995304, 9744995311, 0)","LINESTRING (496541.756 4720170.475, 496521.473...",2,20787
20788,"(9744995304, 9744995310, 0)","LINESTRING (496541.756 4720170.475, 496558.928...",7,20788
20789,"(9964974606, 9964974609, 0)","LINESTRING (494174.708 4719764.250, 494163.668...",4,20789
20790,"(9964974610, 9964974611, 0)","LINESTRING (494100.379 4719512.527, 494038.489...",2,20790


In [11]:
t1=time.time()
intersection,depth=decreasing_depth_intersections(osm_edges_df,osm_crs,G,cursor,max_depth,depth_step=20,max_delta_time_total=float('inf'),max_delta_time_per_level=float('inf'))
t2=time.time()
print('total : %f'%(t2-t1),depth)

520<depth<=540 :0.131611 
500<depth<=520 :0.103584 
480<depth<=500 :0.180332 
460<depth<=480 :0.294392 
440<depth<=460 :0.478287 
420<depth<=440 :0.857891 
400<depth<=420 :1.039724 
380<depth<=400 :1.481711 
360<depth<=380 :2.134876 
340<depth<=360 :2.388254 
320<depth<=340 :3.188379 
300<depth<=320 :3.449449 
280<depth<=300 :4.056138 
260<depth<=280 :5.218209 
240<depth<=260 :5.229188 
220<depth<=240 :6.355103 
200<depth<=220 :8.142669 
180<depth<=200 :9.393256 
160<depth<=180 :11.993446 
140<depth<=160 :12.816519 
120<depth<=140 :14.571401 
100<depth<=120 :17.887214 
80<depth<=100 :24.285041 
60<depth<=80 :28.824414 
40<depth<=60 :39.187162 
20<depth<=40 :39.750136 
0<depth<=20 :58.215758 
total : 301.659273 1


In [12]:
t1=time.time()
pre_osm_nodes_to_contours={}
for edge,df in intersection.groupby('edge'):
    df=df.sort_values('edge_coordinate')
    u,v=edge[:2]
    if not u in pre_osm_nodes_to_contours:
        pre_osm_nodes_to_contours[u]={df.iloc[0]['id']}
    else:
        pre_osm_nodes_to_contours[u].add(df.iloc[0]['id'])
    if not v in pre_osm_nodes_to_contours:
        pre_osm_nodes_to_contours[v]={df.iloc[-1]['id']}
    else:
        pre_osm_nodes_to_contours[v].add(df.iloc[-1]['id'])
t2=time.time()
print(t2-t1)

24.49580144882202


In [13]:
t1=time.time()
G_osm.remove_edges_from(intersection['edge'].unique())
t2=time.time()
print(t2-t1)

0.041687965393066406


In [14]:
t1=time.time()
CC=list(nx.connected_components(nx.Graph(G_osm)))
pre_CC_to_contours={}
for k,cc in enumerate(CC):
    contours_ids=set()
    for osm_node in cc:
        if osm_node in pre_osm_nodes_to_contours:
            contours_ids=contours_ids.union(pre_osm_nodes_to_contours[osm_node])
    pre_CC_to_contours[k]=contours_ids

t2=time.time()
print(t2-t1)

0.31925249099731445


In [15]:
t1=time.time()
CC_to_contours={}
ambiguous_CC={}
for k,contour_ids in pre_CC_to_contours.items():
    if len(contour_ids)>1:
        id_1,id_2=list(contour_ids)[:2]
        if id_2 in G.neighbors(id_1):
            CC_to_contours[k]=id_1
        elif id_1 in G.neighbors(id_2):
            CC_to_contours[k]=id_2
        else:
            predecessors=list(G.predecessors(id_2))
            if len(predecessors)>0:
                CC_to_contours[k]=predecessors[0]
            else:
                print('component is outside with %is point'%len(list(CC[k])))

    else:
        ambiguous_CC[k]=list(contour_ids)[0]

t2=time.time()
print(t2-t1)

0.050158023834228516


In [16]:
contour_id_bunch_size=20

t1=time.time()
ambiguous_CC=pd.DataFrame([{'id_cc':k,'ambiguous_id_contour':v} for k,v in ambiguous_CC.items()])
ambiguous_contours_ids=ambiguous_CC['ambiguous_id_contour'].unique()
for i in range(0,len(ambiguous_contours_ids),contour_id_bunch_size):
    t3=time.time()
    ambiguous_contours_df=get_nodes_data(cursor,ambiguous_contours_ids[i:min(i+contour_id_bunch_size,len(ambiguous_contours_ids))])
    ambiguous_contours_df=ambiguous_contours_df.set_index('id')
    ambiguous_contours_df=ambiguous_contours_df.to_crs(osm_crs)
    ambiguous_contours_df['geometry']=ambiguous_contours_df['geometry'].apply(lambda ls:Polygon(ls))
    t4=time.time()
    for ambiguous_id_contour,row in ambiguous_contours_df.iterrows():
        polygon=row['geometry']
        for id_cc in ambiguous_CC[ambiguous_CC.ambiguous_id_contour==ambiguous_id_contour]['id_cc']:
            osm_node=random.choice(list(CC[id_cc]))
            datum=G_osm.nodes()[osm_node]
            pt=Point(datum['x'],datum['y'])
            if pt.within(polygon):
                CC_to_contours[id_cc]=ambiguous_id_contour
            else:
                predecessors=list(G.predecessors(ambiguous_id_contour))
                if len(predecessors)>0:
                    CC_to_contours[id_cc]=predecessors[0]
                else:
                    print('component is outside with %is point'%len(list(CC[id_cc])))
    t5=time.time()
    print('%i/%i : getting df : %f, check intersections : %f'%(i,len(ambiguous_contours_ids),t4-t3,t5-t4))

t2=time.time()
print(t2-t1)

0/834 : getting df : 4.754120, check intersections : 47.334004
20/834 : getting df : 1.136546, check intersections : 7.171312
40/834 : getting df : 3.779729, check intersections : 6.638018
60/834 : getting df : 4.750259, check intersections : 13.999630
80/834 : getting df : 2.586862, check intersections : 1.197609
100/834 : getting df : 2.772825, check intersections : 6.107169
120/834 : getting df : 1.769719, check intersections : 2.729059
140/834 : getting df : 4.341958, check intersections : 3.488390
160/834 : getting df : 1.659279, check intersections : 0.938064
180/834 : getting df : 3.458516, check intersections : 1.393765
200/834 : getting df : 5.335205, check intersections : 5.507624
220/834 : getting df : 2.512529, check intersections : 3.315049
240/834 : getting df : 4.117526, check intersections : 3.661670
260/834 : getting df : 2.947351, check intersections : 1.073222
280/834 : getting df : 2.856153, check intersections : 2.700490
300/834 : getting df : 3.964451, check inter

In [17]:
len(CC_to_contours),len(CC)

(10864, 10866)

In [18]:
t1=time.time()
inside_osm_nodes_data,outside_osm_nodes_data={},{}
for k,cc in enumerate(CC):
    if k in CC_to_contours:
        for osm_node in cc:
            inside_osm_nodes_data[osm_node]={'contour_line_id':CC_to_contours[k],'geometry':Point(G_osm.nodes()[osm_node]['lon'],G_osm.nodes()[osm_node]['lat'])}
    else:
        for osm_node in cc:
            outside_osm_nodes_data[osm_node]={'geometry':Point(G_osm.nodes()[osm_node]['lon'],G_osm.nodes()[osm_node]['lat'])}
        
t2=time.time()
print(t2-t1)

0.13228321075439453


In [19]:
for elem in outside_osm_nodes_data.values():
    pt=elem['geometry']
    print(pt.y,pt.x)

42.1120529 8.6879432
42.261892 8.6772551


In [20]:
ADD_OSM_DATA=True
if ADD_OSM_DATA:
    
    t1=time.time()
    string_list=',\n'.join(['(%i,%i,ST_GeomFromText(\'%s\'))'%(osm_id,d['contour_line_id'],d['geometry'].wkt) for osm_id,d in inside_osm_nodes_data.items()])
    cmd="INSERT INTO %s (`osm_id`,`contour_line_id`,`geometry`) VALUES %s;"%(osm_nodes_table_name,string_list)
    cursor.execute(cmd)

    string_list=',\n'.join(['(%i,ST_GeomFromText(\'%s\'))'%(osm_id,d['geometry'].wkt) for osm_id,d in outside_osm_nodes_data.items()])
    cmd="INSERT INTO %s (`osm_id`,`geometry`) VALUES %s;"%(osm_nodes_table_name,string_list)
    cursor.execute(cmd)
    
    intersection=intersection.to_crs('epsg:4326')
    string_list=',\n'.join(['(%i,%i,%i,%i,%f,%f,ST_GeomFromText(\'%s\'))'%(row['id'],row['edge'][0],row['edge'][1],row['edge'][2],row['edge_coordinate'],row['elevation'],row['geometry'].wkt)  for _,row in intersection.iterrows()])
    cmd="INSERT INTO %s (`contour_line_id`,`osm_begin`,`osm_end`,`osm_key`,`edge_coordinate`,`elevation`,`geometry`) VALUES %s;"%(osm_intersections_table_name,string_list)
    cursor.execute(cmd)
    t2=time.time()
    print(t2-t1)

10.5498046875
